In [23]:
import urllib.request, json, datetime, time,pytz
import requests
from requests.structures import CaseInsensitiveDict
with open('keys.json') as json_file:
    keys = json.load(json_file)
trainkey = keys["trainkey"]
buskey = keys["buskey"]

In [24]:
# pacestops=[{"route":"226","routeID":"17","dirID":"4","stopID":"21001","stopname":"Oakton & Lincoln"},
#     {"route":"226","routeID":"17","dirID":"1","stopID":"20844","stopname":"Oakton & Lincoln"},
#     {"route":"210","routeID":"10","dirID":"2","stopID":"35464","stopname":"Oakton & Lacrosse"},
#     {"route":"210", "routeID":"10","dirID":"3","stopID":"35469","stopname":"Oakton & Lacrosse"}]

pacestops_origin = [{"route":"293","routeID":"293","dirID":"3","stopID":"22064","stopname":"Milwaukee & Devon"}]

In [97]:
def paceparser(nowlocal, stop):
    url = "http://tmweb.pacebus.com/TMWebWatch/Arrivals.aspx/getStopTimes"
    headers = CaseInsensitiveDict()
    headers["Content-Type"] = "application/json"
        
    minutes = []
    stopdata = '{"routeID": '+stop["routeID"]+',	"directionID": '+stop["dirID"]+',	"stopID":	'+stop["stopID"]+',	"tpID":	0, "useArrivalTimes":	true}'
    resp = requests.post(url, headers=headers, data=stopdata)
    data = resp.json()['d']
    estimations = data['routeStops'][0]['stops'][0]['crossings']
    buses = []
    for est in estimations:
        bus = {}
        bus["stpid"] = stop["stopID"]
        bus["dir"] = stop["dirID"]
        bus['rt'] = stop["route"]
        bus['stpnm'] = stop["stopname"]
        if est['predTime']==None:
            bus['predTime'] = est['schedTime']
            bus['predPeriod'] = est['schedPeriod']
        else:
            bus['predTime'] = est['predTime']
            bus['predPeriod'] = est['predPeriod']
        today_string = nowlocal.strftime("%Y-%m-%d")
        esttimestring = today_string + "T" + bus["predTime"]+bus["predPeriod"]
        est_td = datetime.datetime.strptime(esttimestring,"%Y-%m-%dT%I:%M%p")
        local_tz = pytz.timezone("America/Chicago")
        est_td = local_tz.localize(est_td)
        ETA = est_td - nowlocal
        bus['ETA'] = round(ETA.total_seconds()/60)
        buses.append(bus)
    return buses

In [98]:
nowlocal = datetime.datetime.now()
local_tz =  pytz.timezone("America/Chicago")
nowlocal = local_tz.localize(nowlocal)
paceparser(nowlocal, pacestops_origin[0])

[{'stpid': '22064',
  'dir': '3',
  'rt': '293',
  'predTime': '2:24',
  'predPeriod': 'pm',
  'ETA': 4},
 {'stpid': '22064',
  'dir': '3',
  'rt': '293',
  'predTime': '2:38',
  'predPeriod': 'pm',
  'ETA': 18},
 {'stpid': '22064',
  'dir': '3',
  'rt': '293',
  'predTime': '2:53',
  'predPeriod': 'pm',
  'ETA': 33}]